# 사고 설명문 구조화 with LLM

### colab 세팅

In [1]:
from google.colab import drive
drive.mount('/content/drive')

parent_path = '/content/drive/Othercomputers/My MacBook Pro/colab/'

import sys
sys.path.append(parent_path + 'preprocess')

Mounted at /content/drive


In [2]:
!python '/content/drive/Othercomputers/My MacBook Pro/colab/settings.py'
%env OLLAMA_MAX_LOADED_MODELS=2
%env OLLAMA_NUM_PARALLEL=2
!nohup ollama serve &

Executing command: ['sudo', 'apt-get', 'install', '-y', 'pciutils']
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpci3 pci.ids
The following NEW packages will be installed:
  libpci3 pci.ids pciutils
0 upgraded, 3 newly installed, 0 to remove and 29 not upgraded.
Need to get 343 kB of archives.
After this operation, 1,581 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 pci.ids all 0.0~2022.01.22-1 [251 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpci3 amd64 1:3.7.0-6 [28.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 pciutils amd64 1:3.7.0-6 [63.6 kB]
Fetched 343 kB in 1s (265 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debc

### 구조화 요청 prompt 준비

In [3]:
import pandas as pd
from utils import load_ollama, run_ollama, process_dataframe_in_chunks
import random

test_df = pd.read_csv(parent_path + 'data/test_02.csv')
train_df = pd.read_csv(parent_path + 'data/train_02.csv')

In [4]:
def make_structuring_prompt(row):
    prompt = f"""한국 건설 공사 안전 사고 관련 문장을 언어 모델에 사용하기 위해 전처리하려 합니다.
사고가 발생한 배경과 핵심적인 피해 내용, 사고 원인을 도출하고, \"발생 배경, 사고 종류, 사고 원인\"을 json 형식으로 핵심적인 내용만 간결히 정리해주세요.
추가적인 정보를 임의로 추론하거나 생성하지 말고, 원문에 주어진 정보만을 반영하세요.
제공될 문장은 [문장: \"사고 종류 (사고 설명 또는 원인)\"] 형식으로 제공됩니다.
문장: \"{row['correct_description']}\""""
    return prompt
    # '피해 내용' 삭제
def model_query_test(df, prompt_func, chain, idx=-1):
    if idx == -1:
        random_idx = random.randint(0, len(df) - 1)
    else:
        random_idx = idx
    print(f"idx: {random_idx}")

    prompt = prompt_func(df.iloc[random_idx])
    print(prompt)

    if df.loc[random_idx, 'category_exists'] == "N":
        print("사고 종류 없으므로 그대로 사용.")
        return

    answer = run_ollama(prompt, chain)
    print(answer)

chain = load_ollama(model="gemma3:4b", temperature=0.0)

train_df['structuring_prompt'] = train_df.apply(lambda x: make_structuring_prompt(x), axis=1)
test_df['structuring_prompt'] = test_df.apply(lambda x: make_structuring_prompt(x), axis=1)

### 쿼리 테스트

In [ ]:
model_query_test(train_df, make_structuring_prompt, chain, idx=-1) #21459

idx: 5668
한국 건설 공사 안전 사고 관련 문장을 언어 모델에 사용하기 위해 전처리하려 합니다.
사고가 발생한 배경과 핵심적인 피해 내용, 사고 원인을 도출하고, "발생 배경: , 사고 종류: , 사고 원인: " 형식으로 핵심적인 내용만 간결히 정리해주세요.
추가적인 정보를 임의로 추론하거나 생성하지 말고, 원문에 주어진 정보만을 반영하세요.
제공될 문장은 [문장: "사고 종류 (사고 설명 또는 원인)"] 형식으로 제공됩니다.
문장: "설치작업 중 비계 관련 사고(작업자 부주의)"


발생 배경: 설치작업 중
사고 종류: 비계 관련 사고 (작업자 부주의)
사고 원인: 작업자 부주의


### 모델 준비 및 실행

In [5]:
# 모델 준비
chain1 = load_ollama(model="gemma3:4b", temperature=0.0)
chain2 = load_ollama(model="gemma3:4bcp", temperature=0.0)

In [6]:
no_category_test_df = test_df[test_df['category_exists']=="Y"]
no_category_train_df = train_df[train_df['category_exists']=="Y"]

In [7]:
process_dataframe_in_chunks(no_category_test_df, 'structuring_prompt', 'structured_json', chain1, chain2)
test_df.loc[test_df['category_exists']=="Y", 'structured_json'] = no_category_test_df['structured_json']
test_df.to_csv(parent_path + 'data/test_03.csv', index=False)

/content/drive/Othercomputers/My MacBook Pro/colab/preprocess/utils.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[output_col] = np.nan


처리할 행의 개수: 958, 한번에 연산되는 양: 30개


  0%|          | 0/32 [00:00<?, ?it/s]/content/drive/Othercomputers/My MacBook Pro/colab/preprocess/utils.py:49: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['```json\n{\n  "발생 배경": "타설 작업 중 콘크리트 펌프 사용",\n  "사고 종류": "콘크리트 펌프 관련 부딪힘 및 전도 사고",\n  "사고 원인": "지반 침하 (아웃트리거 우측 상부 1개소), 아웃트리거 펼친 길이 상이, 타설 위치 건물 끝부분 모서리, 붐대호스 최대 펼침으로 인한 장비 무게 중심 불균형"\n}\n```'
 '```json\n{\n  "발생 배경": "절단 작업 중",\n  "사고 종류": "공구류 관련 절단, 베임 사고",\n  "사고 원인": "작업자의 불안전한 행동(숫돌 측면 사용) 및 보안면 미 착용"\n}\n```'
 '```json\n{\n  "발생 배경": "작업자가 작업을 위해 이동 중",\n  "사고 종류": "이동 중 떨어짐 사고",\n  "사고 원인": "작업자가 전방을 주시하지 않아 발을 헛디뎌 계단에서 굴러 넘어짐"\n}\n```'
 '```json\n{\n  "발생 배경": "작업 발판 위 벽돌 잔재",\n  "사고 종류": "넘어짐 (물체에 걸림)",\n  "사고 원인": "작업 발판 위 벽돌 잔재를 밟고 넘어짐"\n}\n```\n'
 '```json\n{\n  "발생 배경": "해체 작업 중",\n  "사고 종류": "떨어짐 사고 (점심 식사 시간 이동 중 작업자 부주의로 인한 추락 사고)",\n  "사고 원인": "작업자 부주의"\n}\n```'
 '```json\n{\n  "발생 배경": "설치작업 중",\n  "사고 종류": "거푸집 관련 물체에 충돌",\n 

In [16]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [7]:
process_dataframe_in_chunks(no_category_train_df, 'structuring_prompt', 'structured_json', chain1, chain2, chunk_size=30)
train_df.loc[train_df['category_exists']=="Y", 'structured_json'] = no_category_train_df['structured_json']
train_df.to_csv(parent_path + 'data/train_03.csv', index=False)

처리할 행의 개수: 11949, 한번에 연산되는 양: 30개


100%|██████████| 399/399 [1:44:30<00:00, 15.72s/it]
